In [3]:
# model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import numpy as np

class ConvAutoencoder:
  @staticmethod
  def build(width, height, depth, filters=(32, 64), latent_dim=16):

    # initialize input shape to be channels last along with channels dimension itself
    input_shape = (height, width, depth)
    chan_dim = -1

    # define input to encoder
    inputs = Input(shape=input_shape)
    x = inputs

    # loop over the number of filters
    for f in filters:
      # apply CONV => RELU => BN operations
      x = Conv2D(f, (3, 3), strides=2, padding='same')(x)
      x = LeakyReLU(alpha=0.2)(x)
      x = BatchNormalization(axis=chan_dim)(x)

    # flatten the network and then consturct our latent vector
    volume_size = K.int_shape(x)
    x = Flatten()(x)
    latent = Dense(latent_dim)(x)

    # build the encoder model
    encoder = Model(inputs, latent, name='encoder')

    # start building the decoder model which will accept output of encoder as its input
    latent_input = Input(shape=(latent_dim,))
    x = Dense(np.prod(volume_size[1:]))(latent_input)
    x = Reshape((volume_size[1], volume_size[2], volume_size[3]))(x)

    # loop over number of filters again but this time in reverse order
    for f in filters[::-1]:
      
      # apply a Conv_transpose => RELU => BN operation
      x = Conv2DTranspose(f, (3, 3), strides=2, padding='same')(x)
      x = LeakyReLU(alpha=0.2)(x)
      x = BatchNormalization(axis=chan_dim)(x)

      # apply a single Conv_Transpose layer used to recover the original depth of image
      x = Conv2DTranspose(depth, (3, 3), padding='same')(x)
      outputs = Activation('sigmoid')(x)

      # build decoder model
      decoder = Model(latent_input, outputs, name='decoder')

      # autoencoder is encoder + decoder
      autoencoder = Model(inputs, decoder(encoder(inputs)), name='autoencoder')

      return (encoder, decoder, autoencoder)